# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
# !pip install -q -U langchain openai

In [2]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [3]:
# !pip install -q -U wikipedia

In [14]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs=100, # YOUR CODE HERE, 
    doc_content_chars_max=100000 ### YOUR CODE HERE
    ).load()

barbie_csv_docs = CSVLoader(
    file_path="./barbie_data/barbie.csv", ### YOUR CODE HERE, 
    source_column="Review_Url" ### YOUR CODE HERE
    ).load()

/Users/bnayak/miniconda3/envs/llmops-week1/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/bnayak/miniconda3/envs/llmops-week1/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [15]:
print(f' Barbie wikipedia length = {len(barbie_wikipedia_docs)}, Barbie csv length = {len(barbie_csv_docs)}')

 Barbie wikipedia length = 9, Barbie csv length = 125


Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, ### YOUR CODE HERE,
    chunk_overlap = 100, ### YOUR CODE HERE,
    length_function = len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ['\n==', '\n', ' '] ### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, ### YOUR CODE HERE,
    chunk_overlap = 100, ### YOUR CODE HERE,
    length_function = len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ['\n', ' '] ### YOUR CODE HERE # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs) ### YOUR CODE HERE
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs) ### YOUR CODE HERE

In [17]:
print(len(chunked_barbie_wikipedia_docs))
print(len(chunked_barbie_csv_docs))

357
181


#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [11]:
# !pip install -q -U rank_bm25 tiktoken faiss-cpu

In [18]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache/")### YOUR CODE HERE

core_embeddings_model = OpenAIEmbeddings() ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model ### YOUR CODE HERE
)
 ### YOUR CODE HERE

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [19]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever() ### YOUR CODE HERE

There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [21]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
   chunked_barbie_wikipedia_docs ### YOUR CODE HERE
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
# barbie_wikipedia_faiss_store = FAISS.from_documents(
#     ### YOUR CODE HERE,
#     ### YOUR CODE HERE
# )
# barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_csv_faiss_retriever, barbie_wikipedia_bm25_retriever], ### YOUR CODE HERE,
    weights= [0.5, 0.5]### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [22]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    retriever = barbie_wikipedia_bm25_retriever, ### YOUR CODE HERE 
    name = "Barbie_Wiki", ### YOUR CODE HERE
    description= "Useful to Look up facts about the Barbie movie on Wikipedia"### YOUR CODE HERE
)

barbie_csv_retrieval_tool = create_retriever_tool(
    retriever = barbie_csv_faiss_retriever,
    name = "Barbie_Reviews",
    description = "Useful to look up reviews about the Barbie movie in csv files"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]### YOUR CODE HERE

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [23]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True) # YOUR CODE HERE

In [24]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `Barbie_Reviews` with `Barbie`


[Document(page_content=": 122\nReview_Date: 30 July 2023\nAuthor: walfordior\nRating: 9\nReview_Title: This is more than just a 'lighthearted comedy'\nReview: Barbie is no longer just a toy; rather, this idea of 'Barbie' has evolved into one of the most famous and significant topics in the world at this moment in time. It is so much more than merely a toy. It will not longer be just 'a toy'\nReview_Url: /review/rw9199947/?ref_=tt_urv", metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 122}), Document(page_content='Barbie mainly for her character. Her speech was moving and she brought so much to her simple, yet relatable mom.', metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 119}), Document(page_content=': 64\nReview_Date: 22 July 2023\nAuthor: fernandoschiavi\nRating: 5\nReview_Title: "Barbie" is fun and visually beautiful, but unfortunately Barbie doll was dragged into the cultu

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Barbie_Reviews', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=": 122\\nReview_Date: 30 July 2023\\nAuthor: walfordior\\nRating: 9\\nReview_Title: This is more than just a \'lighthearted comedy\'\\nReview: Barbie is no longer just a toy; rather, this idea of \'Barbie\' has evolved into one of the most famous and significant topics in the world at this moment in time. It is so much more than merely a toy. It will not longer be just \'a toy\'\\nReview_Url: /review/rw9199947/?ref_=tt_urv", metadata={\'source\': \'/rev

In [25]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Barbie_Wiki` with `Barbie`


[Document(page_content='== Notable designers ==\nKitty Black Perkins, creator of Black Barbie\nCarol Spencer, Barbie fashion designer from 1963-1999\n\n\n== See also ==\nBarbie (media franchise)\nBarbenheimer\nCreatable World\nKitty Black Perkins\nLammily - a crowd funded alternative developed by Nickolay Lamm\nList of Barbie animated films\nList of Barbie video games\nMy Scene\nSindy\nSuperstar: The Karen Carpenter Story\nThe Most Popular Girls in School\nTotally Hair Barbie\n\n\n== References ==\n\n\n== Notes ==', metadata={'title': 'Barbie', 'summary': 'Barbie is a fashion doll created by American businesswoman Ruth Handler, manufactured by American toy company Mattel and launched in 1959. The toy is the figurehead of the Barbie brand that includes a range of fashion dolls and accessories. Barbie has been an important part of the toy fashion doll market for over six decades. Mattel has sold over a billi

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Barbie_Reviews', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=": 122\\nReview_Date: 30 July 2023\\nAuthor: walfordior\\nRating: 9\\nReview_Title: This is more than just a \'lighthearted comedy\'\\nReview: Barbie is no longer just a toy; rather, this idea of \'Barbie\' has evolved into one of the most famous and significant topics in the world at this moment in time. It is so much more than merely a toy. It will not longer be just \'a toy\'\\nReview_Url: /review/rw9199947/?ref_=tt_urv", metadata={\'source\': \'/review/rw9199947/?ref_=tt_urv\', \'row\': 122}), Document(page_cont

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [26]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query="Oppenheimer (film)", 
    load_max_docs=100, # YOUR CODE HERE, 
    doc_content_chars_max=100000 ### YOUR CODE HERE
    ).load() ### YOUR CODE HERE

oppenheimer_csv_docs = CSVLoader(
    file_path="./oppenheimer_data/oppenheimer.csv", ### YOUR CODE HERE, 
    source_column="Review_Url" ### YOUR CODE HERE
    ).load()### YOUR CODE HERE

/Users/bnayak/miniconda3/envs/llmops-week1/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/bnayak/miniconda3/envs/llmops-week1/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [27]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs)### YOUR CODE HERE
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs)### YOUR CODE HERE

In [28]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder).as_retriever() ### YOUR CODE HERE

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_opp_wikipedia_docs) ### YOUR CODE HERE
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, embedder
) ### YOUR CODE HERE
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights=[0.50, 0.50]
) ### YOUR CODE HERE

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [29]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [30]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [31]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content='People generally thought highly of the Oppenheimer movie. It received critical acclaim and was praised for its compelling storytelling, engaging performances, and stunning cinematography. Reviewers described it as a "masterpiece," "magnificent," and "one of the best films of the 21st century." The cast, including Cillian Murphy and Robert Downey Jr., was lauded for their phenomenal performances. Some reviewers mentioned that the film may have lacked emotional depth and missed certain aspects of Oppenheimer\'s life and relationships, but overall, the movie was well-received.', additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [34]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name ='BarbieTool', ### YOUR CODE HERE,
        func=query_oppenheimer,### YOUR CODE HERE,
        description='Useful for answering question about the Barbie movie'### YOUR CODE HERE
    ),
    Tool(
        name = 'OppenheimerTool', ### YOUR CODE HERE,
        func= query_oppenheimer, ### YOUR CODE HERE,
        description= 'Useful for answering question about the Oppenheimer movie' ### YOUR CODE HERE
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [35]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools=tools,### YOUR CODE HERE
    prefix=prefix,### YOUR CODE HERE
    suffix=suffix,### YOUR CODE HERE
    input_variables=["input", "agent_scratchpad"]### YOUR CODE HERE
)

In [36]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt) ### YOUR CODE HERE

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [37]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain,### YOUR AGENTS, 
    tools=tools,### YOUR AGENTS, 
    verbose=True)### YOUR AGENTS)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent,### YOUR AGENTS, 
    tools=tools,### YOUR AGENTS, 
    verbose=True)### YOUR AGENTS)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [38]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Barbie movie, so I should use the BarbieTool to find out what people liked about it.
Action: BarbieTool
Action Input: "What did people like about the Barbie movie?"
Observation: content='There is no specific information available about what people liked about the Barbie movie in the provided sources.' additional_kwargs={} example=False
Thought:I couldn't find any specific information about what people liked about the Barbie movie. I should try using the OppenheimerTool instead to see if it has any relevant information.
Action: OppenheimerTool
Action Input: "What did people like about the Barbie movie?"
Observation: content='There is no specific information available about what people liked about the Barbie movie in the provided sources.' additional_kwargs={} example=False
Thought:

OutputParserException: Could not parse LLM output: `I couldn't find any specific information about what people liked about the Barbie movie using either tool. I don't have enough information to provide a final answer to the question.`

In [39]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have enough information to answer this question.
Action: OppenheimerTool
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content='Based on the information from the sources, people liked several aspects of the Oppenheimer movie. Here are some of the positive points mentioned:\n\n1. Compelling and engaging storytelling: The movie was praised for making the complicated story of J. Robert Oppenheimer compelling and easy to understand.\n\n2. Phenomenal performances: The actors, including Cillian Murphy and Robert Downey Jr., were praised for their phenomenal performances.\n\n3. Beautiful cinematography: The cinematography was described as beautifully crafted, adding to the overall enjoyment of the film.\n\n4. Historical interest: The movie sparked an interest in history, encouraging viewers to learn more about the subject.\n\n5. Emotional impact: While some reviewers felt that the movie lacked emotion, o

'People liked several aspects of the Oppenheimer movie, including its compelling storytelling, phenomenal performances, beautiful cinematography, historical interest, emotional impact, dialogue and character drama, and sense of satisfaction.'

In [40]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find out if the movies Barbie and Oppenheimer share similar themes or ideas.
Action: BarbieTool
Action Input: "similar themes or ideas between Barbie and Oppenheimer"
Observation: content="According to the information provided, both Barbie and Oppenheimer explore similar themes and ideas. They both delve into existentialism and the theoretical notion of the Anthropocene, which is the geological epoch defined by humanity's impact. Additionally, both films are said to examine the corruptibility of men and the relationship between imagination and reality. Both films have also been analyzed as exploring the core ideas of the 20th century, such as accelerating militarism and unbounded consumption." additional_kwargs={} example=False
Thought:Based on the information provided by the BarbieTool, it seems that both Barbie and Oppenheimer share similar themes and ideas. They both explore existentialism, the concept of the Anthropocene, t

'Yes, the movies Barbie and Oppenheimer share similar themes and ideas.'

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: